<a href="https://colab.research.google.com/github/AhmadKElsayed/YouTubeChecker/blob/main/YoutubeChecker_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
[YouTube URL] → [Extract Transcript]
                      ↓
                  [NER Pipeline]
                      ↓
               [Extract Facts / Claims]
                      ↓
             [Fact-check with Gemini]
                      ↓
         [Optionally: Google Search / SerpAPI]
                      ↓
       [Return verdict + explanation + URL]
'''


In [ ]:
!pip install google-generativeai
!pip install youtube-transcript-api
!pip install langchain
!pip install langchain-google-genai
!pip install transformers
# !pip install serpapi
!pip install dateparser
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install -U langchain-community
!pip install duckduckgo-search
!pip install google-search-results

In [ ]:
import os
import dateparser
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.utilities import SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
import spacy
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool

In [ ]:
os.environ["GOOGLE_API_KEY"] = ""
video_url = "https://www.youtube.com/watch?v=JIbIYCM48to&t=99s"

In [ ]:
def get_youtube_transcript(video_url):

    video_id = video_url.split("v=")[-1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    text = " ".join([item['text'] for item in transcript])

    return text
text = get_youtube_transcript(video_url)
print(text)

In [ ]:
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

# Load spaCy and Hugging Face NER model
nlp = spacy.load("en_core_web_sm")

from transformers import pipeline
import spacy
import dateparser

# Load models
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")
nlp = spacy.load("en_core_web_sm")

def extract_factual_claims(text):
    # Process the transcript into sentences
    doc = nlp(text)
    extracted_claims = []

    for sent in doc.sents:
        sentence_text = sent.text.strip()
        if len(sentence_text) < 5:
            continue

        # Run NER on each sentence individually
        entities = ner_pipeline(sentence_text)

        # Only keep sentences with detected entities
        if entities:
            claim_info = {
                "sentence": sentence_text,
                "entities": []
            }
            for ent in entities:
                entity_data = {
                    "text": ent['word'],
                    "type": ent['entity_group'],
                    "score": ent['score']
                }
                if ent['entity_group'] == "DATE":
                    parsed_date = dateparser.parse(ent['word'])
                    entity_data["parsed_date"] = str(parsed_date) if parsed_date else None
                claim_info["entities"].append(entity_data)
            extracted_claims.append(claim_info)

    return extracted_claims


# Example
claims = extract_factual_claims(text)
for c in claims[:5]:
    print(c)


# AGENT:

##LLM:

In [ ]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash-lite-preview", temperature=0.2)

## Tools:

In [ ]:
Youtube_Transcriptor = Tool(name="YouTube Transcript",func=get_youtube_transcript,description="Fetch transcript from a YouTube video given its video URL")

In [ ]:
search = DuckDuckGoSearchRun()
DDSearch = Tool(name="Search", func=search.run, description="Useful for looking up facts or credible URLs"  )

In [ ]:
fact_claim_extractor = Tool(
    name="FactClaimExtractor",
    func=lambda text: str(extract_factual_claims(text)),  # return as stringified JSON for LLM
    description="Extracts factual claims and named entities from a given text transcript. Expects a plain text transcript."
)

In [ ]:
tools = [
    fact_claim_extractor,
    DDSearch
]

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
def fact_check_claim(transcript):

    prompt = f"""
    Fact-check this transcript: "{transcript}"
    First, extract any factual claims or named entities using FactClaimExtractor.
    Then, fact-check those claims one by one.
    - Respond with 'Correct', 'Incorrect', or 'Unverifiable'.
    - Explain briefly why.
    - Get me three credible URL on your own.
    - If you couldn't find them on your own, call the Search tool to look up credible sources and then respond.
    """
    result = agent.run(prompt)
    return result

result = fact_check_claim("Paris is in Italy. Napoleon was born there")
print(result)

In [ ]:
# Example Usage
video_url = "https://www.youtube.com/watch?v=JIbIYCM48to&t=99s"
transcript = get_youtube_transcript(video_url)
facts = extract_factual_claims(transcript)

for fact in facts[:5]:
    print(f"📝 Fact: {fact}")
    result = fact_check_claim(fact)
    print(result)
    print("-" * 50)
    time.sleep(5)  # wait 5 seconds between calls

In [ ]:
from google.generativeai import GenerativeModel
import google.generativeai # Import the main library

# List available models
# models = GenerativeModel.list() # Incorrect way to list models
models = google.generativeai.list_models() # Correct way to list models
for m in models:
    print(m.name, m.description)